In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
import time
import sklearn.preprocessing

pd.set_option("display.max_columns", None)
warnings.filterwarnings(action='ignore')

serviceKey='bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ=='

p7000 ={'serviceKey' :serviceKey,
             'routeId':'200000112','stationId':'228000703'}
class Bus():
    def __init__(self,params):
        self.params=params
        self.bus_time=[]
        
    def repet(self):
        url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
        response = requests.get(url, params=self.params)
        soup=BeautifulSoup(response.text,'xml')
        return soup

        
    def current_buses(self): 
        url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
        mod_params ={'serviceKey' : self.params['serviceKey'],
                  'routeId':self.params['routeId']}

        response=requests.get(url,params=mod_params)
        soup=BeautifulSoup(response.text,'xml')
        locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
        return locations


def trim_data(df,FileName):
    df['시간'] = df['시간'].apply(pd.to_datetime)
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['second']=df['시간'].dt.second
    df['day'] = df['시간'].dt.weekday 

    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
    
    batch=df.index[df.arrived==1]
    df=df.iloc[:batch[-1]+1,:]
    #7/12 -----------------------------------
    batch=df.index[df.arrived==1]
    temp=0
    for x in batch:
        for y in range(temp,x): 
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0
    # 7/12 -----------------------------------

    df = df.set_index('시간')
    df.to_csv(f'./{FileName}')        # 동일 폴더에 파일이 저장될 거임
    return df


In [2]:

past=4
pBus=Bus(p7000)
for x in range(360): # 시간 설정
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'7000_8.9.csv') # 이름 설정
mod_df

0 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0]]
1 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0]]
2 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 78], 0]]
3 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 78], 0], ['2022-08-09 11:14:33.286', [10, 19, 26, 27, 33, 45, 55, 65, 70, 79], 0]]
4 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 7

15 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 78], 0], ['2022-08-09 11:14:33.286', [10, 19, 26, 27, 33, 45, 55, 65, 70, 79], 0], ['2022-08-09 11:15:33.509', [1, 11, 20, 26, 27, 33, 45, 55, 66, 71], 1], ['2022-08-09 11:16:35.247', [2, 12, 21, 26, 28, 34, 46, 56, 68, 73], 0], ['2022-08-09 11:17:35.878', [3, 12, 21, 26, 28, 34, 46, 57, 68, 75], 0], ['2022-08-09 11:18:39.929', [3, 13, 21, 26, 28, 35, 47, 58, 68, 77], 0], ['2022-08-09 11:19:41.075', [4, 14, 22, 27, 28, 36, 48, 59, 69, 78], 0], ['2022-08-09 11:20:41.222', [5, 14, 23, 27, 29, 36, 48, 60, 69, 79], 0], ['2022-08-09 11:21:41.415', [6, 15, 24, 27, 29, 38, 48, 60, 70], 0], ['2022-08-09 11:22:41.624', [7, 16, 24, 28, 29, 38, 49, 61, 73], 0], ['2022-08-09 11:23:44.957', [8, 16, 25, 28, 30, 39, 49, 61, 73], 0], ['2022-08-09 11:24:45.137', [8, 17, 25, 28, 31, 3

22 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 78], 0], ['2022-08-09 11:14:33.286', [10, 19, 26, 27, 33, 45, 55, 65, 70, 79], 0], ['2022-08-09 11:15:33.509', [1, 11, 20, 26, 27, 33, 45, 55, 66, 71], 1], ['2022-08-09 11:16:35.247', [2, 12, 21, 26, 28, 34, 46, 56, 68, 73], 0], ['2022-08-09 11:17:35.878', [3, 12, 21, 26, 28, 34, 46, 57, 68, 75], 0], ['2022-08-09 11:18:39.929', [3, 13, 21, 26, 28, 35, 47, 58, 68, 77], 0], ['2022-08-09 11:19:41.075', [4, 14, 22, 27, 28, 36, 48, 59, 69, 78], 0], ['2022-08-09 11:20:41.222', [5, 14, 23, 27, 29, 36, 48, 60, 69, 79], 0], ['2022-08-09 11:21:41.415', [6, 15, 24, 27, 29, 38, 48, 60, 70], 0], ['2022-08-09 11:22:41.624', [7, 16, 24, 28, 29, 38, 49, 61, 73], 0], ['2022-08-09 11:23:44.957', [8, 16, 25, 28, 30, 39, 49, 61, 73], 0], ['2022-08-09 11:24:45.137', [8, 17, 25, 28, 31, 3

27 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 78], 0], ['2022-08-09 11:14:33.286', [10, 19, 26, 27, 33, 45, 55, 65, 70, 79], 0], ['2022-08-09 11:15:33.509', [1, 11, 20, 26, 27, 33, 45, 55, 66, 71], 1], ['2022-08-09 11:16:35.247', [2, 12, 21, 26, 28, 34, 46, 56, 68, 73], 0], ['2022-08-09 11:17:35.878', [3, 12, 21, 26, 28, 34, 46, 57, 68, 75], 0], ['2022-08-09 11:18:39.929', [3, 13, 21, 26, 28, 35, 47, 58, 68, 77], 0], ['2022-08-09 11:19:41.075', [4, 14, 22, 27, 28, 36, 48, 59, 69, 78], 0], ['2022-08-09 11:20:41.222', [5, 14, 23, 27, 29, 36, 48, 60, 69, 79], 0], ['2022-08-09 11:21:41.415', [6, 15, 24, 27, 29, 38, 48, 60, 70], 0], ['2022-08-09 11:22:41.624', [7, 16, 24, 28, 29, 38, 49, 61, 73], 0], ['2022-08-09 11:23:44.957', [8, 16, 25, 28, 30, 39, 49, 61, 73], 0], ['2022-08-09 11:24:45.137', [8, 17, 25, 28, 31, 3

31 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 78], 0], ['2022-08-09 11:14:33.286', [10, 19, 26, 27, 33, 45, 55, 65, 70, 79], 0], ['2022-08-09 11:15:33.509', [1, 11, 20, 26, 27, 33, 45, 55, 66, 71], 1], ['2022-08-09 11:16:35.247', [2, 12, 21, 26, 28, 34, 46, 56, 68, 73], 0], ['2022-08-09 11:17:35.878', [3, 12, 21, 26, 28, 34, 46, 57, 68, 75], 0], ['2022-08-09 11:18:39.929', [3, 13, 21, 26, 28, 35, 47, 58, 68, 77], 0], ['2022-08-09 11:19:41.075', [4, 14, 22, 27, 28, 36, 48, 59, 69, 78], 0], ['2022-08-09 11:20:41.222', [5, 14, 23, 27, 29, 36, 48, 60, 69, 79], 0], ['2022-08-09 11:21:41.415', [6, 15, 24, 27, 29, 38, 48, 60, 70], 0], ['2022-08-09 11:22:41.624', [7, 16, 24, 28, 29, 38, 49, 61, 73], 0], ['2022-08-09 11:23:44.957', [8, 16, 25, 28, 30, 39, 49, 61, 73], 0], ['2022-08-09 11:24:45.137', [8, 17, 25, 28, 31, 3

...

72 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 78], 0], ['2022-08-09 11:14:33.286', [10, 19, 26, 27, 33, 45, 55, 65, 70, 79], 0], ['2022-08-09 11:15:33.509', [1, 11, 20, 26, 27, 33, 45, 55, 66, 71], 1], ['2022-08-09 11:16:35.247', [2, 12, 21, 26, 28, 34, 46, 56, 68, 73], 0], ['2022-08-09 11:17:35.878', [3, 12, 21, 26, 28, 34, 46, 57, 68, 75], 0], ['2022-08-09 11:18:39.929', [3, 13, 21, 26, 28, 35, 47, 58, 68, 77], 0], ['2022-08-09 11:19:41.075', [4, 14, 22, 27, 28, 36, 48, 59, 69, 78], 0], ['2022-08-09 11:20:41.222', [5, 14, 23, 27, 29, 36, 48, 60, 69, 79], 0], ['2022-08-09 11:21:41.415', [6, 15, 24, 27, 29, 38, 48, 60, 70], 0], ['2022-08-09 11:22:41.624', [7, 16, 24, 28, 29, 38, 49, 61, 73], 0], ['2022-08-09 11:23:44.957', [8, 16, 25, 28, 30, 39, 49, 61, 73], 0], ['2022-08-09 11:24:45.137', [8, 17, 25, 28, 31, 3

74 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 78], 0], ['2022-08-09 11:14:33.286', [10, 19, 26, 27, 33, 45, 55, 65, 70, 79], 0], ['2022-08-09 11:15:33.509', [1, 11, 20, 26, 27, 33, 45, 55, 66, 71], 1], ['2022-08-09 11:16:35.247', [2, 12, 21, 26, 28, 34, 46, 56, 68, 73], 0], ['2022-08-09 11:17:35.878', [3, 12, 21, 26, 28, 34, 46, 57, 68, 75], 0], ['2022-08-09 11:18:39.929', [3, 13, 21, 26, 28, 35, 47, 58, 68, 77], 0], ['2022-08-09 11:19:41.075', [4, 14, 22, 27, 28, 36, 48, 59, 69, 78], 0], ['2022-08-09 11:20:41.222', [5, 14, 23, 27, 29, 36, 48, 60, 69, 79], 0], ['2022-08-09 11:21:41.415', [6, 15, 24, 27, 29, 38, 48, 60, 70], 0], ['2022-08-09 11:22:41.624', [7, 16, 24, 28, 29, 38, 49, 61, 73], 0], ['2022-08-09 11:23:44.957', [8, 16, 25, 28, 30, 39, 49, 61, 73], 0], ['2022-08-09 11:24:45.137', [8, 17, 25, 28, 31, 3

76 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 78], 0], ['2022-08-09 11:14:33.286', [10, 19, 26, 27, 33, 45, 55, 65, 70, 79], 0], ['2022-08-09 11:15:33.509', [1, 11, 20, 26, 27, 33, 45, 55, 66, 71], 1], ['2022-08-09 11:16:35.247', [2, 12, 21, 26, 28, 34, 46, 56, 68, 73], 0], ['2022-08-09 11:17:35.878', [3, 12, 21, 26, 28, 34, 46, 57, 68, 75], 0], ['2022-08-09 11:18:39.929', [3, 13, 21, 26, 28, 35, 47, 58, 68, 77], 0], ['2022-08-09 11:19:41.075', [4, 14, 22, 27, 28, 36, 48, 59, 69, 78], 0], ['2022-08-09 11:20:41.222', [5, 14, 23, 27, 29, 36, 48, 60, 69, 79], 0], ['2022-08-09 11:21:41.415', [6, 15, 24, 27, 29, 38, 48, 60, 70], 0], ['2022-08-09 11:22:41.624', [7, 16, 24, 28, 29, 38, 49, 61, 73], 0], ['2022-08-09 11:23:44.957', [8, 16, 25, 28, 30, 39, 49, 61, 73], 0], ['2022-08-09 11:24:45.137', [8, 17, 25, 28, 31, 3

78 번째 시도:  [['2022-08-09 11:11:31.832', [9, 18, 26, 26, 31, 44, 53, 63, 68, 76], 0], ['2022-08-09 11:12:31.962', [9, 18, 26, 27, 32, 44, 53, 64, 69, 77], 0], ['2022-08-09 11:13:32.109', [10, 19, 26, 27, 32, 45, 54, 65, 69, 78], 0], ['2022-08-09 11:14:33.286', [10, 19, 26, 27, 33, 45, 55, 65, 70, 79], 0], ['2022-08-09 11:15:33.509', [1, 11, 20, 26, 27, 33, 45, 55, 66, 71], 1], ['2022-08-09 11:16:35.247', [2, 12, 21, 26, 28, 34, 46, 56, 68, 73], 0], ['2022-08-09 11:17:35.878', [3, 12, 21, 26, 28, 34, 46, 57, 68, 75], 0], ['2022-08-09 11:18:39.929', [3, 13, 21, 26, 28, 35, 47, 58, 68, 77], 0], ['2022-08-09 11:19:41.075', [4, 14, 22, 27, 28, 36, 48, 59, 69, 78], 0], ['2022-08-09 11:20:41.222', [5, 14, 23, 27, 29, 36, 48, 60, 69, 79], 0], ['2022-08-09 11:21:41.415', [6, 15, 24, 27, 29, 38, 48, 60, 70], 0], ['2022-08-09 11:22:41.624', [7, 16, 24, 28, 29, 38, 49, 61, 73], 0], ['2022-08-09 11:23:44.957', [8, 16, 25, 28, 30, 39, 49, 61, 73], 0], ['2022-08-09 11:24:45.137', [8, 17, 25, 28, 31, 3

ConnectionError: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /6410000/busarrivalservice/getBusArrivalItem?serviceKey=bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ%3D%3D&routeId=200000112&stationId=228000703 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000186BE9F12E0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [3]:
past=4
pBus=Bus(p7000)
for x in range(180): # 시간 설정
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'7000_8.9-1.csv') # 이름 설정
mod_df

0 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0]]
1 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0]]
2 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0]]
3 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0], ['2022-08-09 15:24:24.878', [8, 16, 25, 31, 41, 52, 54, 72], 0]]
4 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0], ['2022-08-09 15:24:24.878', [8, 16, 25, 31, 41, 52, 54, 72], 0], ['2022-08-09 15:25:24.993', [8, 16,

18 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0], ['2022-08-09 15:24:24.878', [8, 16, 25, 31, 41, 52, 54, 72], 0], ['2022-08-09 15:25:24.993', [8, 16, 26, 31, 42, 52, 54, 73], 0], ['2022-08-09 15:26:25.139', [9, 17, 26, 32, 43, 52, 55, 74], 0], ['2022-08-09 15:27:25.332', [9, 18, 26, 32, 44, 52, 56, 74], 0], ['2022-08-09 15:28:25.447', [10, 18, 26, 33, 44, 53, 56, 74], 0], ['2022-08-09 15:29:25.593', [10, 19, 26, 33, 45, 53, 57, 75], 0], ['2022-08-09 15:30:27.208', [1, 11, 19, 27, 34, 45, 53, 58, 76], 1], ['2022-08-09 15:31:35.963', [2, 13, 20, 27, 34, 45, 53, 60, 78], 0], ['2022-08-09 15:32:36.141', [3, 13, 21, 27, 36, 46, 53, 60, 79], 0], ['2022-08-09 15:33:36.318', [3, 14, 21, 28, 37, 46, 53, 61], 0], ['2022-08-09 15:34:36.418', [4, 14, 22, 28, 37, 47, 54, 62], 0], ['2022-08-09 15:35:36.581', [5, 14, 23, 29, 38, 48, 55, 63], 0],

25 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0], ['2022-08-09 15:24:24.878', [8, 16, 25, 31, 41, 52, 54, 72], 0], ['2022-08-09 15:25:24.993', [8, 16, 26, 31, 42, 52, 54, 73], 0], ['2022-08-09 15:26:25.139', [9, 17, 26, 32, 43, 52, 55, 74], 0], ['2022-08-09 15:27:25.332', [9, 18, 26, 32, 44, 52, 56, 74], 0], ['2022-08-09 15:28:25.447', [10, 18, 26, 33, 44, 53, 56, 74], 0], ['2022-08-09 15:29:25.593', [10, 19, 26, 33, 45, 53, 57, 75], 0], ['2022-08-09 15:30:27.208', [1, 11, 19, 27, 34, 45, 53, 58, 76], 1], ['2022-08-09 15:31:35.963', [2, 13, 20, 27, 34, 45, 53, 60, 78], 0], ['2022-08-09 15:32:36.141', [3, 13, 21, 27, 36, 46, 53, 60, 79], 0], ['2022-08-09 15:33:36.318', [3, 14, 21, 28, 37, 46, 53, 61], 0], ['2022-08-09 15:34:36.418', [4, 14, 22, 28, 37, 47, 54, 62], 0], ['2022-08-09 15:35:36.581', [5, 14, 23, 29, 38, 48, 55, 63], 0],

30 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0], ['2022-08-09 15:24:24.878', [8, 16, 25, 31, 41, 52, 54, 72], 0], ['2022-08-09 15:25:24.993', [8, 16, 26, 31, 42, 52, 54, 73], 0], ['2022-08-09 15:26:25.139', [9, 17, 26, 32, 43, 52, 55, 74], 0], ['2022-08-09 15:27:25.332', [9, 18, 26, 32, 44, 52, 56, 74], 0], ['2022-08-09 15:28:25.447', [10, 18, 26, 33, 44, 53, 56, 74], 0], ['2022-08-09 15:29:25.593', [10, 19, 26, 33, 45, 53, 57, 75], 0], ['2022-08-09 15:30:27.208', [1, 11, 19, 27, 34, 45, 53, 58, 76], 1], ['2022-08-09 15:31:35.963', [2, 13, 20, 27, 34, 45, 53, 60, 78], 0], ['2022-08-09 15:32:36.141', [3, 13, 21, 27, 36, 46, 53, 60, 79], 0], ['2022-08-09 15:33:36.318', [3, 14, 21, 28, 37, 46, 53, 61], 0], ['2022-08-09 15:34:36.418', [4, 14, 22, 28, 37, 47, 54, 62], 0], ['2022-08-09 15:35:36.581', [5, 14, 23, 29, 38, 48, 55, 63], 0],

35 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0], ['2022-08-09 15:24:24.878', [8, 16, 25, 31, 41, 52, 54, 72], 0], ['2022-08-09 15:25:24.993', [8, 16, 26, 31, 42, 52, 54, 73], 0], ['2022-08-09 15:26:25.139', [9, 17, 26, 32, 43, 52, 55, 74], 0], ['2022-08-09 15:27:25.332', [9, 18, 26, 32, 44, 52, 56, 74], 0], ['2022-08-09 15:28:25.447', [10, 18, 26, 33, 44, 53, 56, 74], 0], ['2022-08-09 15:29:25.593', [10, 19, 26, 33, 45, 53, 57, 75], 0], ['2022-08-09 15:30:27.208', [1, 11, 19, 27, 34, 45, 53, 58, 76], 1], ['2022-08-09 15:31:35.963', [2, 13, 20, 27, 34, 45, 53, 60, 78], 0], ['2022-08-09 15:32:36.141', [3, 13, 21, 27, 36, 46, 53, 60, 79], 0], ['2022-08-09 15:33:36.318', [3, 14, 21, 28, 37, 46, 53, 61], 0], ['2022-08-09 15:34:36.418', [4, 14, 22, 28, 37, 47, 54, 62], 0], ['2022-08-09 15:35:36.581', [5, 14, 23, 29, 38, 48, 55, 63], 0],

...

91 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0], ['2022-08-09 15:24:24.878', [8, 16, 25, 31, 41, 52, 54, 72], 0], ['2022-08-09 15:25:24.993', [8, 16, 26, 31, 42, 52, 54, 73], 0], ['2022-08-09 15:26:25.139', [9, 17, 26, 32, 43, 52, 55, 74], 0], ['2022-08-09 15:27:25.332', [9, 18, 26, 32, 44, 52, 56, 74], 0], ['2022-08-09 15:28:25.447', [10, 18, 26, 33, 44, 53, 56, 74], 0], ['2022-08-09 15:29:25.593', [10, 19, 26, 33, 45, 53, 57, 75], 0], ['2022-08-09 15:30:27.208', [1, 11, 19, 27, 34, 45, 53, 58, 76], 1], ['2022-08-09 15:31:35.963', [2, 13, 20, 27, 34, 45, 53, 60, 78], 0], ['2022-08-09 15:32:36.141', [3, 13, 21, 27, 36, 46, 53, 60, 79], 0], ['2022-08-09 15:33:36.318', [3, 14, 21, 28, 37, 46, 53, 61], 0], ['2022-08-09 15:34:36.418', [4, 14, 22, 28, 37, 47, 54, 62], 0], ['2022-08-09 15:35:36.581', [5, 14, 23, 29, 38, 48, 55, 63], 0],

94 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0], ['2022-08-09 15:24:24.878', [8, 16, 25, 31, 41, 52, 54, 72], 0], ['2022-08-09 15:25:24.993', [8, 16, 26, 31, 42, 52, 54, 73], 0], ['2022-08-09 15:26:25.139', [9, 17, 26, 32, 43, 52, 55, 74], 0], ['2022-08-09 15:27:25.332', [9, 18, 26, 32, 44, 52, 56, 74], 0], ['2022-08-09 15:28:25.447', [10, 18, 26, 33, 44, 53, 56, 74], 0], ['2022-08-09 15:29:25.593', [10, 19, 26, 33, 45, 53, 57, 75], 0], ['2022-08-09 15:30:27.208', [1, 11, 19, 27, 34, 45, 53, 58, 76], 1], ['2022-08-09 15:31:35.963', [2, 13, 20, 27, 34, 45, 53, 60, 78], 0], ['2022-08-09 15:32:36.141', [3, 13, 21, 27, 36, 46, 53, 60, 79], 0], ['2022-08-09 15:33:36.318', [3, 14, 21, 28, 37, 46, 53, 61], 0], ['2022-08-09 15:34:36.418', [4, 14, 22, 28, 37, 47, 54, 62], 0], ['2022-08-09 15:35:36.581', [5, 14, 23, 29, 38, 48, 55, 63], 0],

96 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0], ['2022-08-09 15:24:24.878', [8, 16, 25, 31, 41, 52, 54, 72], 0], ['2022-08-09 15:25:24.993', [8, 16, 26, 31, 42, 52, 54, 73], 0], ['2022-08-09 15:26:25.139', [9, 17, 26, 32, 43, 52, 55, 74], 0], ['2022-08-09 15:27:25.332', [9, 18, 26, 32, 44, 52, 56, 74], 0], ['2022-08-09 15:28:25.447', [10, 18, 26, 33, 44, 53, 56, 74], 0], ['2022-08-09 15:29:25.593', [10, 19, 26, 33, 45, 53, 57, 75], 0], ['2022-08-09 15:30:27.208', [1, 11, 19, 27, 34, 45, 53, 58, 76], 1], ['2022-08-09 15:31:35.963', [2, 13, 20, 27, 34, 45, 53, 60, 78], 0], ['2022-08-09 15:32:36.141', [3, 13, 21, 27, 36, 46, 53, 60, 79], 0], ['2022-08-09 15:33:36.318', [3, 14, 21, 28, 37, 46, 53, 61], 0], ['2022-08-09 15:34:36.418', [4, 14, 22, 28, 37, 47, 54, 62], 0], ['2022-08-09 15:35:36.581', [5, 14, 23, 29, 38, 48, 55, 63], 0],

98 번째 시도:  [['2022-08-09 15:21:17.189', [5, 14, 24, 29, 40, 51, 53, 70], 0], ['2022-08-09 15:22:17.555', [6, 15, 24, 29, 41, 51, 53, 71], 0], ['2022-08-09 15:23:24.716', [7, 15, 25, 30, 41, 51, 53, 71], 0], ['2022-08-09 15:24:24.878', [8, 16, 25, 31, 41, 52, 54, 72], 0], ['2022-08-09 15:25:24.993', [8, 16, 26, 31, 42, 52, 54, 73], 0], ['2022-08-09 15:26:25.139', [9, 17, 26, 32, 43, 52, 55, 74], 0], ['2022-08-09 15:27:25.332', [9, 18, 26, 32, 44, 52, 56, 74], 0], ['2022-08-09 15:28:25.447', [10, 18, 26, 33, 44, 53, 56, 74], 0], ['2022-08-09 15:29:25.593', [10, 19, 26, 33, 45, 53, 57, 75], 0], ['2022-08-09 15:30:27.208', [1, 11, 19, 27, 34, 45, 53, 58, 76], 1], ['2022-08-09 15:31:35.963', [2, 13, 20, 27, 34, 45, 53, 60, 78], 0], ['2022-08-09 15:32:36.141', [3, 13, 21, 27, 36, 46, 53, 60, 79], 0], ['2022-08-09 15:33:36.318', [3, 14, 21, 28, 37, 46, 53, 61], 0], ['2022-08-09 15:34:36.418', [4, 14, 22, 28, 37, 47, 54, 62], 0], ['2022-08-09 15:35:36.581', [5, 14, 23, 29, 38, 48, 55, 63], 0],

ConnectionError: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /6410000/busarrivalservice/getBusArrivalItem?serviceKey=bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ%3D%3D&routeId=200000112&stationId=228000703 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000186C053A670>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [4]:
past=4
pBus=Bus(p7000)
for x in range(180): # 시간 설정
    soup=pBus.repet()
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'7000_8.9-2.csv') # 이름 설정
mod_df

0 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1]]
1 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0]]
2 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 0]]
3 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 0], ['2022-08-09 19:24:23.204', [5, 11, 24, 26, 32, 37, 40, 46, 50, 60], 0]]
4 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 

15 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 0], ['2022-08-09 19:24:23.204', [5, 11, 24, 26, 32, 37, 40, 46, 50, 60], 0], ['2022-08-09 19:25:23.430', [6, 11, 24, 26, 33, 37, 41, 47, 50, 61], 0], ['2022-08-09 19:26:33.481', [7, 11, 25, 26, 33, 38, 41, 48, 50, 62], 0], ['2022-08-09 19:27:33.627', [8, 12, 25, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:28:33.757', [8, 13, 26, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:29:33.951', [9, 13, 26, 27, 36, 39, 42, 49, 50, 63], 0], ['2022-08-09 19:30:37.394', [9, 13, 26, 27, 36, 39, 42, 49, 50, 64], 0], ['2022-08-09 19:31:37.680', [10, 13, 26, 27, 37, 40, 43, 49, 51, 65], 0], ['2022-08-09 19:32:43.154', [10, 13, 26, 27, 38, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:33:43.363', [11, 14, 26, 28, 39, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:34:43.493', [12, 14,

21 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 0], ['2022-08-09 19:24:23.204', [5, 11, 24, 26, 32, 37, 40, 46, 50, 60], 0], ['2022-08-09 19:25:23.430', [6, 11, 24, 26, 33, 37, 41, 47, 50, 61], 0], ['2022-08-09 19:26:33.481', [7, 11, 25, 26, 33, 38, 41, 48, 50, 62], 0], ['2022-08-09 19:27:33.627', [8, 12, 25, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:28:33.757', [8, 13, 26, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:29:33.951', [9, 13, 26, 27, 36, 39, 42, 49, 50, 63], 0], ['2022-08-09 19:30:37.394', [9, 13, 26, 27, 36, 39, 42, 49, 50, 64], 0], ['2022-08-09 19:31:37.680', [10, 13, 26, 27, 37, 40, 43, 49, 51, 65], 0], ['2022-08-09 19:32:43.154', [10, 13, 26, 27, 38, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:33:43.363', [11, 14, 26, 28, 39, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:34:43.493', [12, 14,

26 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 0], ['2022-08-09 19:24:23.204', [5, 11, 24, 26, 32, 37, 40, 46, 50, 60], 0], ['2022-08-09 19:25:23.430', [6, 11, 24, 26, 33, 37, 41, 47, 50, 61], 0], ['2022-08-09 19:26:33.481', [7, 11, 25, 26, 33, 38, 41, 48, 50, 62], 0], ['2022-08-09 19:27:33.627', [8, 12, 25, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:28:33.757', [8, 13, 26, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:29:33.951', [9, 13, 26, 27, 36, 39, 42, 49, 50, 63], 0], ['2022-08-09 19:30:37.394', [9, 13, 26, 27, 36, 39, 42, 49, 50, 64], 0], ['2022-08-09 19:31:37.680', [10, 13, 26, 27, 37, 40, 43, 49, 51, 65], 0], ['2022-08-09 19:32:43.154', [10, 13, 26, 27, 38, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:33:43.363', [11, 14, 26, 28, 39, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:34:43.493', [12, 14,

30 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 0], ['2022-08-09 19:24:23.204', [5, 11, 24, 26, 32, 37, 40, 46, 50, 60], 0], ['2022-08-09 19:25:23.430', [6, 11, 24, 26, 33, 37, 41, 47, 50, 61], 0], ['2022-08-09 19:26:33.481', [7, 11, 25, 26, 33, 38, 41, 48, 50, 62], 0], ['2022-08-09 19:27:33.627', [8, 12, 25, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:28:33.757', [8, 13, 26, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:29:33.951', [9, 13, 26, 27, 36, 39, 42, 49, 50, 63], 0], ['2022-08-09 19:30:37.394', [9, 13, 26, 27, 36, 39, 42, 49, 50, 64], 0], ['2022-08-09 19:31:37.680', [10, 13, 26, 27, 37, 40, 43, 49, 51, 65], 0], ['2022-08-09 19:32:43.154', [10, 13, 26, 27, 38, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:33:43.363', [11, 14, 26, 28, 39, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:34:43.493', [12, 14,

...

69 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 0], ['2022-08-09 19:24:23.204', [5, 11, 24, 26, 32, 37, 40, 46, 50, 60], 0], ['2022-08-09 19:25:23.430', [6, 11, 24, 26, 33, 37, 41, 47, 50, 61], 0], ['2022-08-09 19:26:33.481', [7, 11, 25, 26, 33, 38, 41, 48, 50, 62], 0], ['2022-08-09 19:27:33.627', [8, 12, 25, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:28:33.757', [8, 13, 26, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:29:33.951', [9, 13, 26, 27, 36, 39, 42, 49, 50, 63], 0], ['2022-08-09 19:30:37.394', [9, 13, 26, 27, 36, 39, 42, 49, 50, 64], 0], ['2022-08-09 19:31:37.680', [10, 13, 26, 27, 37, 40, 43, 49, 51, 65], 0], ['2022-08-09 19:32:43.154', [10, 13, 26, 27, 38, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:33:43.363', [11, 14, 26, 28, 39, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:34:43.493', [12, 14,

71 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 0], ['2022-08-09 19:24:23.204', [5, 11, 24, 26, 32, 37, 40, 46, 50, 60], 0], ['2022-08-09 19:25:23.430', [6, 11, 24, 26, 33, 37, 41, 47, 50, 61], 0], ['2022-08-09 19:26:33.481', [7, 11, 25, 26, 33, 38, 41, 48, 50, 62], 0], ['2022-08-09 19:27:33.627', [8, 12, 25, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:28:33.757', [8, 13, 26, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:29:33.951', [9, 13, 26, 27, 36, 39, 42, 49, 50, 63], 0], ['2022-08-09 19:30:37.394', [9, 13, 26, 27, 36, 39, 42, 49, 50, 64], 0], ['2022-08-09 19:31:37.680', [10, 13, 26, 27, 37, 40, 43, 49, 51, 65], 0], ['2022-08-09 19:32:43.154', [10, 13, 26, 27, 38, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:33:43.363', [11, 14, 26, 28, 39, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:34:43.493', [12, 14,

73 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 0], ['2022-08-09 19:24:23.204', [5, 11, 24, 26, 32, 37, 40, 46, 50, 60], 0], ['2022-08-09 19:25:23.430', [6, 11, 24, 26, 33, 37, 41, 47, 50, 61], 0], ['2022-08-09 19:26:33.481', [7, 11, 25, 26, 33, 38, 41, 48, 50, 62], 0], ['2022-08-09 19:27:33.627', [8, 12, 25, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:28:33.757', [8, 13, 26, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:29:33.951', [9, 13, 26, 27, 36, 39, 42, 49, 50, 63], 0], ['2022-08-09 19:30:37.394', [9, 13, 26, 27, 36, 39, 42, 49, 50, 64], 0], ['2022-08-09 19:31:37.680', [10, 13, 26, 27, 37, 40, 43, 49, 51, 65], 0], ['2022-08-09 19:32:43.154', [10, 13, 26, 27, 38, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:33:43.363', [11, 14, 26, 28, 39, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:34:43.493', [12, 14,

75 번째 시도:  [['2022-08-09 19:21:22.777', [2, 10, 21, 26, 31, 35, 40, 45, 49, 58], 1], ['2022-08-09 19:22:22.907', [3, 10, 21, 26, 31, 36, 40, 45, 50, 59], 0], ['2022-08-09 19:23:23.053', [4, 11, 22, 26, 32, 36, 40, 46, 50, 60], 0], ['2022-08-09 19:24:23.204', [5, 11, 24, 26, 32, 37, 40, 46, 50, 60], 0], ['2022-08-09 19:25:23.430', [6, 11, 24, 26, 33, 37, 41, 47, 50, 61], 0], ['2022-08-09 19:26:33.481', [7, 11, 25, 26, 33, 38, 41, 48, 50, 62], 0], ['2022-08-09 19:27:33.627', [8, 12, 25, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:28:33.757', [8, 13, 26, 26, 34, 38, 41, 48, 50, 63], 0], ['2022-08-09 19:29:33.951', [9, 13, 26, 27, 36, 39, 42, 49, 50, 63], 0], ['2022-08-09 19:30:37.394', [9, 13, 26, 27, 36, 39, 42, 49, 50, 64], 0], ['2022-08-09 19:31:37.680', [10, 13, 26, 27, 37, 40, 43, 49, 51, 65], 0], ['2022-08-09 19:32:43.154', [10, 13, 26, 27, 38, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:33:43.363', [11, 14, 26, 28, 39, 41, 44, 49, 51, 65], 0], ['2022-08-09 19:34:43.493', [12, 14,

ConnectionError: HTTPConnectionPool(host='apis.data.go.kr', port=80): Max retries exceeded with url: /6410000/busarrivalservice/getBusArrivalItem?serviceKey=bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ%3D%3D&routeId=200000112&stationId=228000703 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000186C059B7F0>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))